In [2]:
!pip install azure-eventhub

StatementMeta(apachesparkpool, 14, 2, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.13.0 requires Flask<2.3.0,>=1.0.4, but you have flask 2.3.3 which is incompatible.
dash 2.13.0 requires Werkzeug<2.3.0, but you have werkzeug 2.3.7 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.


In [7]:
import requests
import asyncio
import time
import json

from azure.eventhub import EventData
from azure.eventhub.aio import EventHubProducerClient
from datetime import datetime

url = "https://flight-radar1.p.rapidapi.com/flights/"

def call_airline_api():
    function = "search"

    querystring = {"query":"F93055","limit":"25"}

    headers = {
        "X-RapidAPI-Key": "#########################################",
        "X-RapidAPI-Host": "flight-radar1.p.rapidapi.com"
    }

    response = requests.get(url + function, headers=headers, params=querystring)

    res = response.json()

    flight_obj = {
        "id" : res["results"][0]["id"],
        "label" : res["results"][0]["label"],
        "type" : res["results"][0]["type"],
        "callsign" : res["results"][0]["detail"]["callsign"],
        "flight_num" : res["results"][0]["detail"]["flight"]
    }

    # get flight trail data to get latest location for streaming

    function = "detail"

    querystring = {"flight": flight_obj["id"]}

    response = requests.get(url + function, headers=headers, params=querystring)

    flight_obj["latest_trail"] = response.json()["trail"][0]
    flight_obj["latest_trail"]["datetime"] = datetime.fromtimestamp(flight_obj["latest_trail"]["ts"])
    return json.dumps(flight_obj, indent=4, default=str)

StatementMeta(apachesparkpool, 14, 7, Finished, Available)

In [4]:
EVENT_HUB_CONNECTION_STR = "Endpoint=sb://airlinestreamhub.servicebus.windows.net/;SharedAccessKeyName=program_policy;SharedAccessKey=######################s##########AEhO+/33g=;EntityPath=airelineevent"
EVENT_HUB_NAME = "airelineevent"

StatementMeta(apachesparkpool, 14, 4, Finished, Available)

In [5]:
async def run(flight_obj):
    # Create a producer client to send messages to the event hub.
    # Specify a connection string to your event hubs namespace and
    # the event hub name.
    producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENT_HUB_CONNECTION_STR, eventhub_name=EVENT_HUB_NAME
    )
    async with producer:
        # Create a batch.
        event_data_batch = await producer.create_batch()

        # Add events to the batch.
        event_data_batch.add(EventData(flight_obj))
        print("sending...")
        # Send the batch of events to the event hub.
        await producer.send_batch(event_data_batch)
        print("sent!")

StatementMeta(apachesparkpool, 14, 5, Finished, Available)

In [9]:
for x in range(5):
    time.sleep(10) # 2 min
    plane_trail = call_airline_api()
    await run(plane_trail)

StatementMeta(apachesparkpool, 14, 9, Finished, Cancelled)

sending...
sent!
sending...
sent!
